## Graph Neural Network Model for Route Optimization



In [ ]:
!pip install torch
!pip install osmnx
!pip install pandas numpy osmnx geopandas shapely matplotlib scikit-learn pathlib gymnasium torch torch_geometric torch_sparse torch_scatter

In [ ]:
import glob
import warnings
import time
from tqdm import tqdm

import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torch_geometric.data import Data
from torch_geometric.nn import GINEConv

from sklearn.model_selection import train_test_split

from torch.nn import Linear, ReLU, Dropout, LayerNorm, GRU

In [ ]:
warnings.filterwarnings("ignore", message="invalid value encountered in cast", category=RuntimeWarning)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

# === 1) Load & Clean EV CSVs ===
temp_cols = [
    'VehId','Trip','Timestamp(ms)',
    'Latitude[deg]','Longitude[deg]',
    'Vehicle Speed[km/h]','MAF[g/sec]',
    'Engine RPM[RPM]','Absolute Load[%]',
    'Speed Limit[km/h]'
]
dtypes = {'VehId':'int32','Trip':'int32'}
chunksize = 200_000

lat_mins, lat_maxs, lon_mins, lon_maxs = [], [], [], []
_cleaned_frames = []

def load_ev_data(path):
    for chunk in pd.read_csv(
        path,
        usecols=temp_cols,
        dtype=dtypes,
        chunksize=chunksize,
        low_memory=False,
        on_bad_lines='warn'
    ):
        # Convert to numeric, fillna
        for c in temp_cols[2:]:
            if c in chunk:
                num = pd.to_numeric(chunk[c], errors='coerce').fillna(0)
                chunk[c] = num.astype('float32') if c!='Timestamp(ms)' else num.astype('int64')
        # Drop zeros lat/lon
        chunk = chunk[(chunk['Latitude[deg]']!=0)&(chunk['Longitude[deg]']!=0)]
        if chunk.empty:
            continue
        lat_mins.append(chunk['Latitude[deg]'].min())
        lat_maxs.append(chunk['Latitude[deg]'].max())
        lon_mins.append(chunk['Longitude[deg]'].min())
        lon_maxs.append(chunk['Longitude[deg]'].max())
        _cleaned_frames.append(chunk.sort_values(['VehId','Trip','Timestamp(ms)']))

file_paths = glob.glob('./eVED/*.csv')
if not file_paths:
    raise FileNotFoundError("No CSV files found in ./eVED/")
for p in file_paths:
    load_ev_data(p)

ev_df = pd.concat(_cleaned_frames, ignore_index=True)
print(f"Loaded EV data: {len(ev_df)} rows")

LAT_MIN, LAT_MAX = min(lat_mins), max(lat_maxs)
LNG_MIN, LNG_MAX = min(lon_mins), max(lon_maxs)

def normalize_coords(lat, lon):
    nl = (lat - LAT_MIN)/(LAT_MAX - LAT_MIN) if LAT_MAX!=LAT_MIN else 0
    ml = (lon - LNG_MIN)/(LNG_MAX - LNG_MIN) if LNG_MAX!=LNG_MIN else 0
    return np.clip(nl,0,1), np.clip(ml,0,1)

In [ ]:
center_lat, center_lon = (LAT_MIN+LAT_MAX)/2, (LNG_MIN+LNG_MAX)/2
print("Downloading OSMnx graph…")
G_nx = ox.graph_from_point((center_lat, center_lon), dist=30000, network_type='drive')

node_id_map = {nid:i for i,nid in enumerate(G_nx.nodes())}
idx2osm = {i:nid for nid,i in node_id_map.items()}

# Node features: normalized coords
node_feats = []
for nid, data in G_nx.nodes(data=True):
    nl, ml = normalize_coords(data['y'], data['x'])
    node_feats.append([nl, ml])
x = torch.tensor(node_feats, dtype=torch.float)

# Edge features + index
e_u, e_v, edge_feats = [], [], []
lengths, speeds = [], []
for u, v, data in G_nx.edges(data=True):
    lengths.append(data.get('length', 0.0))
    ms = data.get('maxspeed', 0)
    if isinstance(ms, list): ms = ms[0]
    try:
        speeds.append(float(str(ms).split()[0]))
    except:
        speeds.append(0.0)

max_len = max(lengths) or 1.0
MAX_SP = 130.0

for (u, v), length, speed in zip(G_nx.edges(), lengths, speeds):
    ui, vi = node_id_map[u], node_id_map[v]
    e_u.append(ui); e_v.append(vi)
    nl = length / max_len
    ns = np.clip(speed / MAX_SP, 0, 1)
    edge_feats.append([nl, ns])

edge_index = torch.tensor([e_u, e_v], dtype=torch.long)
edge_attr  = torch.tensor(edge_feats, dtype=torch.float)
graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr).to(DEVICE)
print(graph)

# Precompute quick lookup from (osm_u,osm_v) → edge_idx
osm_edge_to_idx = {
    (idx2osm[u], idx2osm[v]): i
    for i, (u, v) in enumerate(zip(edge_index[0].tolist(), edge_index[1].tolist()))
}

In [ ]:
ev_df['node_osm'] = ox.nearest_nodes(
    G_nx,
    X=ev_df['Longitude[deg]'],
    Y=ev_df['Latitude[deg]']
)
ev_df['node_idx'] = ev_df['node_osm'].map(node_id_map)
ev_df.dropna(subset=['node_idx'], inplace=True)
ev_df['node_idx'] = ev_df['node_idx'].astype(int)
ev_df.sort_values(['VehId','Trip','Timestamp(ms)'], inplace=True)

# Compute per-step durations & energy
ev_df['duration_s'] = ev_df.groupby(['VehId','Trip'])['Timestamp(ms)']\
                       .diff().fillna(0) / 1000
ev_df['step_energy'] = ev_df['MAF[g/sec]'] * ev_df['duration_s']

# Collect trips with start, end, true_energy
trips = []
for _, grp in ev_df.groupby(['VehId','Trip']):
    if len(grp) < 2:
        continue
    s = int(grp.iloc[0].node_idx)
    d = int(grp.iloc[-1].node_idx)
    E = grp.step_energy.sum()
    if s != d and E > 0:
        trips.append({'source': s, 'destination': d, 'true_energy': E})

trips_df = pd.DataFrame(trips)
print(f"Processed {len(trips_df)} trips")

In [ ]:
class PathTripDataset(Dataset):
    def __init__(self, trip_df, osm_graph, osm_to_idx, edge_lookup):
        self.trips = trip_df.reset_index(drop=True)
        self.G_nx = osm_graph
        self.idx2osm = idx2osm
        self.edge_lookup = edge_lookup

    def __len__(self):
        return len(self.trips)

    def __getitem__(self, i):
        r = self.trips.iloc[i]
        src_idx = int(r.source)
        dst_idx = int(r.destination)
        os_src = self.idx2osm[src_idx]
        os_dst = self.idx2osm[dst_idx]

        # shortest path of osm IDs
        try:
            path_nodes = nx.shortest_path(
                self.G_nx, os_src, os_dst, weight='length'
            )
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            path_nodes = []

        # map successive node pairs to edge indices
        idxs = []
        for u_osm, v_osm in zip(path_nodes[:-1], path_nodes[1:]):
            ei = self.edge_lookup.get((u_osm, v_osm))
            if ei is not None:
                idxs.append(ei)

        edge_idxs = torch.tensor(idxs, dtype=torch.long)
        energy_log = torch.tensor(np.log1p(r.true_energy), dtype=torch.float32)
        return edge_idxs, energy_log

def collate_paths(batch):
    paths, energies = zip(*batch)
    return list(paths), torch.stack(energies)

# Splits
train_df, val_df = train_test_split(trips_df, test_size=0.15, random_state=42)
train_ds = PathTripDataset(train_df, G_nx, node_id_map, osm_edge_to_idx)
val_ds   = PathTripDataset(val_df,   G_nx, node_id_map, osm_edge_to_idx)

train_loader = DataLoader(
    train_ds, batch_size=16, shuffle=True, collate_fn=collate_paths
)
val_loader = DataLoader(
    val_ds, batch_size=32, shuffle=False, collate_fn=collate_paths
)

In [ ]:
class GNNPathEnergyPredictor(nn.Module):
    def __init__(self, node_dim, edge_dim, hid=128, rnn_hid=128, nlayers=4, drop=0.3):
        super().__init__()
        self.node_emb = Linear(node_dim, hid)
        self.edge_emb = Linear(edge_dim, hid)

        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()
        for _ in range(nlayers):
            mlp = nn.Sequential(Linear(hid, hid), ReLU(), Linear(hid, hid))
            self.convs.append(GINEConv(mlp, edge_dim=hid))
            self.norms.append(LayerNorm(hid))

        self.rnn = GRU(input_size=3*hid, hidden_size=rnn_hid, batch_first=True)
        self.decoder = nn.Sequential(
            Linear(rnn_hid, hid),
            ReLU(),
            Dropout(drop),
            Linear(hid, 1)
        )

    def forward(self, graph, path_list):
        x = self.node_emb(graph.x)
        ea = self.edge_emb(graph.edge_attr)
        # GINE layers
        for conv, norm in zip(self.convs, self.norms):
            res = x
            x = conv(x, graph.edge_index, ea)
            x = norm(x)
            x = F.relu(x) + res
        emb = x

        # build per-path sequences of [src_emb | edge_emb | dst_emb]
        seqs = []
        for eidx in path_list:
            src_nodes = graph.edge_index[0, eidx]
            dst_nodes = graph.edge_index[1, eidx]
            seq = torch.cat([emb[src_nodes], ea[eidx], emb[dst_nodes]], dim=1)
            seqs.append(seq)

        # pack & GRU
        lengths = [s.size(0) for s in seqs]
        padded = nn.utils.rnn.pad_sequence(seqs, batch_first=True)
        packed = nn.utils.rnn.pack_padded_sequence(
            padded, lengths, batch_first=True, enforce_sorted=False
        )
        _, h = self.rnn(packed)
        out = self.decoder(h.squeeze(0)).squeeze(1)
        return out

In [ ]:
model = GNNPathEnergyPredictor(
    node_dim=graph.x.shape[1],
    edge_dim=graph.edge_attr.shape[1]
).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=5e-4)
loss_fn = nn.MSELoss()

best_val = float('inf')
for epoch in range(1, 31):
    t0 = time.time()
    model.train()
    train_loss = 0
    for paths, energies in tqdm(train_loader, desc=f"Epoch {epoch} [train]"):
        opt.zero_grad()
        preds = model(graph, [p.to(DEVICE) for p in paths])
        loss = loss_fn(preds, energies.to(DEVICE))
        loss.backward()
        opt.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for paths, energies in tqdm(val_loader, desc=f"Epoch {epoch} [val]"):
            preds = model(graph, [p.to(DEVICE) for p in paths])
            val_loss += loss_fn(preds, energies.to(DEVICE)).item()
    val_loss /= len(val_loader)

    print(f"Epoch {epoch:02d} | train {train_loss:.4f} | val {val_loss:.4f} | {time.time()-t0:.1f}s")
    if val_loss < best_val:
        best_val = val_loss
        torch.save(model.state_dict(), './best_path_energy_predictor.pth')
        print("  🚀 Saved improved model")

print("Training complete. Best val loss:", best_val)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import numpy as np
import random
import osmnx as ox
import networkx as nx

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tqdm import tqdm

# --- Config & Globals ---
MODEL_PATH       = './best_path_energy_predictor.pth'
DEVICE           = 'cuda' if torch.cuda.is_available() else 'cpu'
SCALING_METHOD   = 'log1p'  # since we used np.log1p in training

# Assumes these exist in your namespace:
#   graph        : torch_geometric.Data  (moved to DEVICE)
#   G_nx         : networkx.Graph       (OSMnx graph_from_point)
#   node_id_map  : dict[OSM ID → idx]
#   val_loader   : DataLoader over validation set, collate_fn returning (paths_list, energy_log_tensor)
#   val_trips_df : DataFrame (val_loader.dataset.trips) with columns ['source','destination','true_energy']

# --- Load & prepare model ---
# Bring your model class into scope
# from your_pipeline_module import GNNPathEnergyPredictor

node_dim = graph.x.shape[1]
edge_dim = graph.edge_attr.shape[1]
model = GNNPathEnergyPredictor(node_dim, edge_dim).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

# --- Inverse scaling helper ---
def inverse_scale(y_log):
    y = y_log.cpu().numpy()
    if SCALING_METHOD=='log1p':
        return np.expm1(y)
    return y

# --- (A) General Performance on Val Set ---
def analyze_general_performance(model, loader):
    all_preds, all_actuals = [], []

    for paths, energy_log in tqdm(loader, desc="Val preds"):
        # forward
        preds_log = model(graph, [p.to(DEVICE) for p in paths])
        # inverse-scale both preds & actuals
        preds = inverse_scale(preds_log)
        actuals = inverse_scale(energy_log)

        all_preds.extend(preds)
        all_actuals.extend(actuals)

    y_pred = np.array(all_preds)
    y_true = np.array(all_actuals)

    # filter out any invalids
    mask = np.isfinite(y_pred) & np.isfinite(y_true)
    y_pred, y_true = y_pred[mask], y_true[mask]

    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2   = r2_score(y_true, y_pred)

    print("\nValidation Metrics (original scale):")
    print(f"  MAE:  {mae:.3f}")
    print(f"  RMSE: {rmse:.3f}")
    print(f"  R²:   {r2:.3f}")

    # scatter
    plt.figure(figsize=(6,6))
    sns.scatterplot(x=y_true, y=y_pred, alpha=0.5, s=10)
    lims = [
        np.min([y_true.min(), y_pred.min()])*0.9,
        np.max([y_true.max(), y_pred.max()])*1.1
    ]
    plt.plot(lims, lims, '--', color='red', label='Ideal')
    plt.xlabel("Actual Energy")
    plt.ylabel("Predicted Energy")
    plt.title(f"Val: MAE={mae:.2f}, RMSE={rmse:.2f}, R²={r2:.2f}")
    plt.legend()
    plt.grid(True)
    plt.xlim(lims); plt.ylim(lims)
    plt.show()

# --- (B) Single-Route Visualization ---
def plot_route_prediction(model, dataset_df, idx2osm, sample_idx):
    # sample_idx is relative to val_trips_df.reset_index(drop=True)
    row = dataset_df.iloc[sample_idx]
    src, dst, E_true = row.source, row.destination, row.true_energy

    # find NX shortest path on OSMnx graph
    os_src = idx2osm[src]; os_dst = idx2osm[dst]
    path_nodes = nx.shortest_path(G_nx, os_src, os_dst, weight='length')

    # build edge-index list
    # use same osm_edge_to_idx you built in pipeline
    edge_list = []
    for u,v in zip(path_nodes[:-1], path_nodes[1:]):
        e = osm_edge_to_idx.get((u,v))
        if e is not None:
            edge_list.append(e)
    edge_tensor = torch.tensor(edge_list, dtype=torch.long).to(DEVICE)

    # predict
    with torch.no_grad():
        pred_log = model(graph, [edge_tensor])
    E_pred = inverse_scale(pred_log).item()

    print(f"\nRoute #{sample_idx}:  Actual={E_true:.1f}, Pred={E_pred:.1f}")
    # plot on map
    fig, ax = ox.plot_graph_route(
        G_nx, path_nodes,
        route_color='r', route_linewidth=2, node_size=0,
        show=False, close=False
    )
    ax.set_title(f"Trip {sample_idx}: Pred={E_pred:.1f} vs Act={E_true:.1f}")
    plt.show()

# --- (C) Weight Histograms ---
def plot_weight_histograms(model, bins=50):
    layers = {
        'node_emb': model.node_emb,
        'edge_emb': model.edge_emb,
        'dec_L1':   model.decoder[0],
        'dec_L2':   model.decoder[3],
    }
    n = len(layers)
    fig, axes = plt.subplots((n+1)//2, 2, figsize=(10, 4*((n+1)//2)))
    axes = axes.flatten()
    for i,(name, layer) in enumerate(layers.items()):
        w = layer.weight.data.cpu().numpy().ravel()
        axes[i].hist(w, bins=bins)
        axes[i].set_title(name)
    plt.tight_layout()
    plt.show()

# --- Run everything ---
if __name__ == "__main__":
    # (1) General metrics + scatter
    analyze_general_performance(model, val_loader)

    # (2) Pick a random validation sample
    val_trips_df = val_loader.dataset.trips
    if len(val_trips_df):
        idx2osm = {v:k for k,v in node_id_map.items()}
        rnd = random.randrange(len(val_trips_df))
        plot_route_prediction(model, val_trips_df, idx2osm, rnd)

    # (3) Weight histograms
    plot_weight_histograms(model)
